# Feature Engineering 2

**Import data from Kaggle**

1. Upload the `kaggle.json` file and get ready to download the SAML-D dataset.

In [ ]:
from google.colab import files
files.upload()

2. This step sets up your Kaggle API credentials in a secure way so you can access Kaggle datasets programmatically

In [ ]:
import os
os.makedirs('/root/.kaggle', exist_ok=True)
!mv kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

3. Download dataset from https://www.kaggle.com/datasets/berkanoztas/synthetic-transaction-monitoring-dataset-aml

In [ ]:
!kaggle datasets download -d berkanoztas/synthetic-transaction-monitoring-dataset-aml

4. This step extracts the contents of a ZIP file using Python's built-in `zipfile` module.

In [ ]:
import zipfile

with zipfile.ZipFile("synthetic-transaction-monitoring-dataset-aml.zip", 'r') as zip_ref:
    zip_ref.extractall("synthetic_transaction_data")

5. Reads the CSV file named `SAML-D.csv` into a Pandas DataFrame called `df`

In [ ]:
import pandas as pd
import polars as pl
os.listdir("synthetic_transaction_data")
df = pl.read_csv("synthetic_transaction_data/SAML-D.csv")
df.head(5)

**Convert `Date` and `Time` to `datetime`**

In [ ]:
df = df.with_columns(
    pl.col("Date").str.strptime(pl.Date, "%Y-%m-%d").alias("Date"),
    pl.col("Time").str.strptime(pl.Time, "%H:%M:%S").alias("Time")
)
df.head()

In [ ]:
df.with_columns(pl.col("Date").cast(pl.Datetime))

**Recast integer-based columns**

In [ ]:
'''
We recast the integer‑based columns following the logic rules outlined in the paper.
"Explainable Feature Engineering for Multi-class Money Laundering Classification"
This recasting is performed to optimize storage efficiency and reduce overall memory consumption."
Excluding "Sender_account" and "Receiver_account" variables.
'''

def recast(df):
    exclude = ['Sender_account', 'Receiver_account']

    for col in df.columns:
        if col not in exclude:
            dtype = df[col].dtype
            if dtype in (pl.Int64, pl.Int32):
              maxval = df[col].max()
              if maxval:
                  if maxval < 127:
                      df = df.with_columns(df[col].cast(pl.Int8).alias(col))
                  elif maxval < 32767:
                      df = df.with_columns(df[col].cast(pl.Int16).alias(col))
                  elif maxval < 2147483647:
                      df = df.with_columns(df[col].cast(pl.Int32).alias(col))

    return df

In [ ]:
df = recast(df)
df.head()

**Temporal Features**

In [ ]:
def temporal_features(df):

    return df.with_columns([
        df["Date"].dt.year().alias("year"),
        df["Date"].dt.month().alias("month"),
        df["Date"].dt.day().alias("day_of_month"),
        df["Date"].dt.weekday().alias("day_of_week"),
        df["Date"].dt.ordinal_day().alias("day_of_year"),
        df["Time"].dt.hour().alias("hour"),
        df["Time"].dt.minute().alias("minute"),
        df["Time"].dt.second().alias("second"),
    ])


In [ ]:
df = temporal_features(df)
df.head()

**Transaction Risk Features**

In [ ]:
high_risk_countries = ['Mexico', 'Turkey', 'Morocco', 'UAE']

In [ ]:
def risk_features(df):

    return df.with_columns([
        (df["Payment_currency"] != df["Received_currency"]).cast(pl.Int8).alias("currency_mismatch"),
        (df["Payment_type"] == "Cross-border").cast(pl.Int8).alias("cross_border"),
        df["Sender_bank_location"].is_in(high_risk_countries).cast(pl.Int8).alias("high_risk_sender"),
        df["Receiver_bank_location"].is_in(high_risk_countries).cast(pl.Int8).alias("high_risk_receiver")])

In [ ]:
df = risk_features(df)
df

In [ ]:
# dropping column Laundering_type
df = df.drop("Laundering_type")

**Rolling window computing**

In [ ]:
import polars as pl

def build_window_features_lazy(
    df,
    specs,
    date_col="Date",
    sender_col="Sender_account",
    receiver_col="Receiver_account",
    amount_col="Amount",
    index_name="__row_idx",
    label_choice="left",
):
    lf = df.lazy() if isinstance(df, pl.DataFrame) else df
    lf = lf.with_columns(pl.arange(0, pl.len()).over(pl.lit(True)).alias(index_name))
    out_lf = lf

    for spec in specs:
        kind = spec.get("kind", "rolling")

        if kind == "rolling":
            # existing rolling logic (no change)
            name = spec["name"]
            direction = spec["type"]  # "fanin" or "fanout"
            period_days = int(spec["period_days"])
            every = spec.get("every", "1d")

            if direction == "fanin":
                group_by = receiver_col
                agg_on = sender_col
            else:
                group_by = sender_col
                agg_on = receiver_col

            win_label = label_choice
            strategy = "forward" if win_label == "left" else "backward"

            right = (
                lf
                .sort([group_by, date_col])
                .group_by_dynamic(
                    index_column=date_col,
                    every=every,
                    period=f"{period_days}d",
                    group_by=group_by,
                    closed="both",
                    label=win_label
                )
                .agg(pl.col(agg_on).n_unique().alias(name))
                .sort([group_by, date_col])
            )

            left = out_lf.sort([group_by, date_col])

            out_lf = left.join_asof(
                right,
                left_on=date_col,
                right_on=date_col,
                by=group_by,
                strategy=strategy,
            )

        elif kind == "monthly":
            # existing monthly logic (no change)
            name = spec["name"]
            side = spec.get("side", "receive")
            group_col = receiver_col if side == "receive" else sender_col

            monthly_agg = (
                lf
                .with_columns(pl.col(date_col).dt.truncate("1mo").alias("__month"))
                .group_by([group_col, "__month"])
                .agg(pl.col(amount_col).sum().alias(name))
            )

            out_lf = (
                out_lf
                .with_columns(pl.col(date_col).dt.truncate("1mo").alias("__month"))
                .join(monthly_agg, on=[group_col, "__month"], how="left")
                .drop("__month")
            )

        elif kind == "daily_pair_count":
            # NEW: back_and_forth_transfers (exact-match on day + pair)
            name = spec["name"]  # e.g., "back_and_forth_transfers"
            # day key = calendar day (truncate to 1 day)
            day_key = "__day"
            # compute counts per sender/receiver/day using lf (lazy)
            pair_daily_agg = (
                lf
                .with_columns(pl.col(date_col).dt.truncate("1d").alias(day_key))
                .group_by([sender_col, receiver_col, day_key])
                .agg(pl.len().alias(name))  # .len() counts rows in group
            )

            # attach day key to working frame and join exact on pair + day
            out_lf = (
                out_lf
                .with_columns(pl.col(date_col).dt.truncate("1d").alias(day_key))
                .join(pair_daily_agg, on=[sender_col, receiver_col, day_key], how="left")
                .fill_null(0)       # optional: replace nulls with 0
                .with_columns(pl.col(name).cast(pl.Int64))  # ensure integer type
                .drop(day_key)
            )

        else:
            raise ValueError("spec kind must be 'rolling', 'monthly', or 'daily_pair_count'")

    return out_lf

In [ ]:
specs = [
    {"name":"fanin_30d", "kind":"rolling", "type":"fanin", "period_days":30, "every":"1d"},
    {"name":"fanout_30d", "kind":"rolling", "type":"fanout", "period_days":30, "every":"1d"},
    {"name":"daily_recieve", "kind":"rolling", "type":"fanin", "period_days":1, "every":"1d"},
    #{"name":"daily_receive", "kind":"daily_unique_count", "side":"receive"},
    {"name":"monthly_receive", "kind":"monthly", "side":"receive"},
    {"name":"monthly_send",    "kind":"monthly", "side":"send"},
    {"name":"back_and_forth_transfers", "kind":"daily_pair_count"},
]
lazy_with_features = build_window_features_lazy(df, specs, amount_col="Amount", label_choice="left")


In [ ]:
plan = (
    lazy_with_features
    .sort(["Receiver_account", "Date"])
    .with_columns([pl.col("Receiver_account").set_sorted(), pl.col("Date").set_sorted()])
)
df_streamed = plan.collect(engine="streaming")
df_final = df_streamed.sort("__row_idx").drop("__row_idx")

In [ ]:
df_final.head()

**More computation**

In [ ]:
from typing import List, Dict

def compute_derived_features_lazy(
    lf: pl.LazyFrame,
    *,
    fanin_col: str = "fanin_30d",
    fanout_col: str = "fanout_30d",
    daily_receive_col: str = "daily_receive",
    monthly_receive_col: str = "monthly_receive",
    monthly_send_col: str = "monthly_send",
    amount_col: str = "Amount",
    sender_col: str = "Sender_account",
    receiver_col: str = "Receiver_account",
    index_name: str = "__row_idx",
) -> pl.LazyFrame:
    """
    Take a LazyFrame and return a LazyFrame with derived features:
      - fan_in_out_ratio (safe division, 0 when denom missing or zero)
      - fanin_intensity_ratio (fanin_30d / daily_receive, 0 when denom missing or zero)
      - amount_dispersion_std (per-sender std of Amount, filled 0 when null)
      - sent_to_received_ratio_monthly (monthly_receive / monthly_send, 0 when denom missing or zero)

    If `daily_receive` is not present in lf.schema(), it is computed lazily as the
    per-receiver unique-senders per calendar day (dt.truncate("1d")) and joined back.
    The function is fully lazy; call .collect(...) when ready.
    """
    # ensure lazy input
    lf = lf if isinstance(lf, pl.LazyFrame) else lf.lazy()

    # Attempt to read schema; if unavailable assume missing and compute
    try:
        schema = lf.schema()
        has_daily = daily_receive_col in schema
    except Exception:
        has_daily = False

    # If daily_receive missing, compute it lazily (exact day bucket of unique senders per receiver)
    if not has_daily:
        day_key = "__day_for_daily_receive"
        daily_receive_agg = (
            lf
            .with_columns(pl.col("Date").dt.truncate("1d").alias(day_key))
            .group_by([receiver_col, day_key])
            .agg(pl.col(sender_col).n_unique().alias(daily_receive_col))
        )
        lf = (
            lf
            .with_columns(pl.col("Date").dt.truncate("1d").alias(day_key))
            .join(daily_receive_agg, on=[receiver_col, day_key], how="left")
            .drop(day_key)
        )

    # safe division helper expression
    def safe_div_expr(num: str, den: str, out_name: str):
        return (
            pl.when(pl.col(den).is_null() | (pl.col(den) == 0))
              .then(0.0)
              .otherwise(pl.col(num).cast(pl.Float64) / pl.col(den).cast(pl.Float64))
              .alias(out_name)
        )

    fan_in_out_expr = safe_div_expr(fanin_col, fanout_col, "fan_in_out_ratio")
    fanin_intensity_expr = safe_div_expr(fanin_col, daily_receive_col, "fanin_intensity_ratio")
    sent_to_received_monthly_expr = safe_div_expr(monthly_receive_col, monthly_send_col, "sent_to_received_ratio_monthly")

    # per-sender std aggregation (lazy) and join back
    sender_std_agg = (
        lf
        .select([sender_col, amount_col])
        .group_by(sender_col)
        .agg(pl.col(amount_col).std().alias("__amount_std"))
    )

    out = (
        lf
        .join(sender_std_agg, on=sender_col, how="left")
        .with_columns(
            pl.col("__amount_std").cast(pl.Float64).fill_null(0.0).alias("amount_dispersion_std")
        )
        .drop("__amount_std")
        .with_columns([
            fan_in_out_expr,
            fanin_intensity_expr,
            sent_to_received_monthly_expr
        ])
    )

    return out

In [ ]:
lazy_with_derived = compute_derived_features_lazy(lazy_with_features)

# Before streaming collect: pick a primary grouping ordering that matches your rolling computations.
# If most rolling features used Receiver_account then Date, use that; otherwise use the grouping you chose.
plan_derived = (
    lazy_with_derived
    .sort(["Receiver_account", "Date"])
    .with_columns([pl.col("Receiver_account").set_sorted(), pl.col("Date").set_sorted()])
)

df_streamed = plan_derived.collect(engine="streaming")
df_final = df_streamed.sort("__row_idx").drop("__row_idx")

In [ ]:
df_final.head()

In [ ]:
def add_daily_weekly_transaction_counts(
    lf: pl.LazyFrame,
    date_col: str = "Date",
    sender_col: str = "Sender_account",
    receiver_col: str = "Receiver_account",
    amount_col: str = "Amount",
) -> pl.LazyFrame:
    """
    Return a LazyFrame with four new columns:
      - daily_receiver_transaction
      - weekly_receiver_transaction
      - daily_sender_transaction
      - weekly_sender_transaction

    The function keeps the pipeline lazy: it returns a LazyFrame you can .collect() later.
    """
    # base lazy frame with truncated calendar buckets
    base = lf.with_columns(
        [
            pl.col(date_col).dt.truncate("1d").alias("_day"),
            pl.col(date_col).dt.truncate("1w").alias("_week"),
        ]
    )

    # aggregations (still lazy)
    agg_daily_recv = (
        base
        .group_by([receiver_col, "_day"])
        .agg(pl.count(amount_col).alias("daily_receiver_transaction"))
    )

    agg_weekly_recv = (
        base
        .group_by([receiver_col, "_week"])
        .agg(pl.count(amount_col).alias("weekly_receiver_transaction"))
    )

    agg_daily_sndr = (
        base
        .group_by([sender_col, "_day"])
        .agg(pl.count(amount_col).alias("daily_sender_transaction"))
    )

    agg_weekly_sndr = (
        base
        .group_by([sender_col, "_week"])
        .agg(pl.count(amount_col).alias("weekly_sender_transaction"))
    )

    # join aggregated counts back to the base lazyframe
    out = (
        base
        .join(agg_daily_recv, left_on=[receiver_col, "_day"], right_on=[receiver_col, "_day"], how="left")
        .join(agg_weekly_recv, left_on=[receiver_col, "_week"], right_on=[receiver_col, "_week"], how="left")
        .join(agg_daily_sndr, left_on=[sender_col, "_day"], right_on=[sender_col, "_day"], how="left")
        .join(agg_weekly_sndr, left_on=[sender_col, "_week"], right_on=[sender_col, "_week"], how="left")
        .with_columns(
            [
                pl.coalesce(["daily_receiver_transaction", pl.lit(0)]).cast(pl.Int64),
                pl.coalesce(["weekly_receiver_transaction", pl.lit(0)]).cast(pl.Int64),
                pl.coalesce(["daily_sender_transaction", pl.lit(0)]).cast(pl.Int64),
                pl.coalesce(["weekly_sender_transaction", pl.lit(0)]).cast(pl.Int64),
            ]
        )
        .drop(["_day", "_week"])
    )

    return out

In [ ]:
df_final_lazy = df_final.lazy()
result_lazy = add_daily_weekly_transaction_counts(df_final_lazy)
df_result = result_lazy.collect(engine="streaming")

In [ ]:
df_result.head()

In [ ]:
df_result = recast(df_result)
df_result.head()

**Circular_transaction_count**

In [ ]:
!pip install rustworkx

In [ ]:
import rustworkx as rx

def circular_count_monthly_rx(pdf, year, month):
    edges = list(zip(pdf["Sender_account"], pdf["Receiver_account"]))
    if not edges:
        return empty_month_frame()

    G = rx.PyDiGraph()
    node_idx = {}
    for u, v in edges:
        if u not in node_idx:
            node_idx[u] = G.add_node(u)
        if v not in node_idx:
            node_idx[v] = G.add_node(v)
        G.add_edge(node_idx[u], node_idx[v], None)

    cycles = rx.simple_cycles(G)

    counter = {}
    for cyc in cycles:
        cyc_nodes = [G[node] for node in cyc]
        for node in cyc_nodes:
            counter[node] = counter.get(node, 0) + 1

    return pl.DataFrame({
        "Sender_account": list(counter.keys()),
        "circular_transaction_count": list(counter.values()),
        "year": [year] * len(counter),
        "month": [month] * len(counter)
    })

In [30]:
# Iterate over monthly groups
results_rx = []
for (year, month), group in df.group_by(["year", "month"]):
    res = circular_count_monthly_rx(group, year, month)
    if res.height > 0:
        results_rx.append(res)

# Combine all results
out_rx = pl.concat(results_rx, how="vertical") if results_rx else pl.DataFrame()

# Join back to original df
df_result = (
    df.join(out_rx, on=["Sender_account", "year", "month"], how="left")
      .with_columns(
          pl.col("circular_transaction_count").fill_null(0)
      )
)

In [31]:
df_result.head()

Time,Date,Sender_account,Receiver_account,Amount,Payment_currency,Received_currency,Sender_bank_location,Receiver_bank_location,Payment_type,Is_laundering,year,month,day_of_month,day_of_week,day_of_year,hour,minute,second,currency_mismatch,cross_border,high_risk_sender,high_risk_receiver,circular_transaction_count
time,date,i64,i64,f64,str,str,str,str,str,i8,i32,i8,i8,i8,i16,i8,i8,i8,i8,i8,i8,i8,i64
10:35:19,2022-10-07,8724731955,2769355426,1459.15,"""UK pounds""","""UK pounds""","""UK""","""UK""","""Cash Deposit""",0,2022,10,7,5,280,10,35,19,0,0,0,0,0
10:35:20,2022-10-07,1491989064,8401255335,6019.64,"""UK pounds""","""Dirham""","""UK""","""UAE""","""Cross-border""",0,2022,10,7,5,280,10,35,20,1,1,0,1,0
10:35:20,2022-10-07,287305149,4404767002,14328.44,"""UK pounds""","""UK pounds""","""UK""","""UK""","""Cheque""",0,2022,10,7,5,280,10,35,20,0,0,0,0,0
10:35:21,2022-10-07,5376652437,9600420220,11895.0,"""UK pounds""","""UK pounds""","""UK""","""UK""","""ACH""",0,2022,10,7,5,280,10,35,21,0,0,0,0,0
10:35:21,2022-10-07,9614186178,3803336972,115.25,"""UK pounds""","""UK pounds""","""UK""","""UK""","""Cash Deposit""",0,2022,10,7,5,280,10,35,21,0,0,0,0,0


In [32]:
df_result.select('circular_transaction_count').describe()

statistic,circular_transaction_count
str,f64
"""count""",9.504852e6
"""null_count""",0.0
"""mean""",0.404003
"""std""",2.012755
"""min""",0.0
"""25%""",0.0
"""50%""",0.0
"""75%""",0.0
"""max""",17.0


In [35]:
df_filtered = df_result.filter(pl.col("circular_transaction_count") >= 1)

In [36]:
df_filtered.height

919026

In [38]:
df_filtered['circular_transaction_count'].describe()

statistic,value
str,f64
"""count""",919026.0
"""null_count""",0.0
"""mean""",4.178325
"""std""",5.11158
"""min""",1.0
"""25%""",1.0
"""50%""",1.0
"""75%""",9.0
"""max""",17.0
